In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def doktor_profil_ozetini_getir(sayfa_no):
    response = requests.get("https://www.doktorsitesi.com/tumuzmanlar?sayfa=" + str(sayfa_no))
    soup = BeautifulSoup(response.text, 'html.parser')
    az_content = soup.find('div', class_='az-content')
    az_main_wrappers = az_content.find_all('div', class_='az-main-wrapper')
    doctorlar = []
    for az_main_wrapper in az_main_wrappers:
        verified = az_main_wrapper.find('div', class_='verified')
        dogrulanmis_profil = False
        if verified is not None:
            dogrulanmis_profil = verified.text
        resim = az_main_wrapper.find('img')
        resim_linki = resim['src']
        cinsiyet = resim['data-gender']
        profil = az_main_wrapper.find('a')
        # <a href="https://www.doktorsitesi.com/ahmet-dincer/psikoloji/istanbul"> <span>Psk.</span> Ahmet Dinçer </a>        
        profil_linki = profil['href']
        konum = profil_linki.split('/')[-1]
        uzmanlik_alani = profil_linki.split('/')[-2]
        # unvanı <span>Psk.</span> olarak alır
        unvan = profil.find('span').text
        # ismi buradan alıp ve temizler \nPsk.\n                                Arzu Kantarcıoğlu\n
        isim = profil.text.split('\n')[2].strip()

        doctorlar.append({
            'resim_linki': resim_linki,
            'unvan': unvan,
            'isim': isim,
            'dogrulanmis_profil': dogrulanmis_profil,
            'profil_linki': profil_linki,
            'cinsiyet': cinsiyet,
            'konum': konum,
            'uzmanlik_alani': uzmanlik_alani
        })

    return doctorlar



In [3]:
tum_doktorlar = []
for i in range(1, 999):
    doktorlar = doktor_profil_ozetini_getir(i)
    tum_doktorlar.extend(doktorlar)

doktorlar_data_frame = pd.DataFrame(
    columns=['resim_linki', 'unvan', 'isim', 'dogrulanmis_profil', 'profil_linki', 'cinsiyet', 'konum', 'uzmanlik_alani'],
    data=tum_doktorlar
)

doktorlar_data_frame.to_csv('tum_uzmanlar.csv', index=False)
doktorlar_data_frame.head()

,resim_linki,unvan,isim,dogrulanmis_profil,profil_linki,cinsiyet,konum,uzmanlik_alani
0,https://www.doktorsitesi.com/media/cache/profi...,Psk.,Arzu Kantarcıoğlu,,https://www.doktorsitesi.com/psk-arzu-kantarci...,female,bursa,psikoloji
1,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Abdullah Atasoy,,https://www.doktorsitesi.com/dr-abdullah-ataso...,male,balikesir,aile-hekimligi
2,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Atakan Saçlı,,https://www.doktorsitesi.com/dr-atakan-sacli/p...,male,izmir,pratisyen-hekimlik
3,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayla Özer(ağar),,https://www.doktorsitesi.com/op-dr-ayla-ozerag...,female,izmir,kadin-hastaliklari-ve-dogum
4,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayşe Gülgün Bilgin,,https://www.doktorsitesi.com/op-dr-ayse-gulgun...,female,mugla,kadin-hastaliklari-ve-dogum
